In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load portfolios (monthly)
p_spy = pd.read_csv("/mnt/data/power_momentum_1R 36 SPY Vol.csv")
p_spy_gld = pd.read_csv("/mnt/data/power_momentum_1R 36 SPY Vol GLD.csv")
spy_m = pd.read_csv("/mnt/data/spy.csv")  # for SPY monthly shock proxy

# Prep dates
for df in (p_spy, p_spy_gld):
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
spy_m['Date'] = pd.to_datetime(spy_m['Date'])
spy_m = spy_m.sort_values('Date')

# Ensure Return column exists (fallback: compute from Balance)
def ensure_return(df):
    if 'Return' in df.columns:
        df['Return'] = pd.to_numeric(df['Return'], errors='coerce')
        return df
    for cand in ['Monthly_Return','Portfolio Return','Result Return','Result_Return','Port Return']:
        if cand in df.columns:
            df['Return'] = pd.to_numeric(df[cand], errors='coerce')
            return df
    # fallback from Balance/Equity
    for cand in ['Balance','End Balance','Equity','Equity Curve']:
        if cand in df.columns:
            df['Return'] = pd.to_numeric(df[cand], errors='coerce').pct_change()
            return df
    raise ValueError("No return-like column found")

p_spy = ensure_return(p_spy)
p_spy_gld = ensure_return(p_spy_gld)

# Limit to 2022+
p_spy = p_spy[p_spy['Date'] >= "2022-01-01"].copy()
p_spy_gld = p_spy_gld[p_spy_gld['Date'] >= "2022-01-01"].copy()

# Compute SPY monthly returns from spy.csv (Result Close is in brackets)
spy_m['Close'] = spy_m['Result Close'].str.strip("[]").astype(float)
spy_m = spy_m[['Date','Close']].copy()
spy_m['SPY_mret'] = spy_m['Close'].pct_change()

# Merge SPY monthly returns to portfolio dates (left join)
p_spy = p_spy.merge(spy_m[['Date','SPY_mret']], on='Date', how='left')
p_spy_gld = p_spy_gld.merge(spy_m[['Date','SPY_mret']], on='Date', how='left')

def add_overlays(df):
    df = df.copy()
    r = df['Return'].fillna(0.0).values
    dates = df['Date'].values
    spy_r = df['SPY_mret'].fillna(0.0).values
    
    # Baseline equity
    eq_base = np.cumprod(1 + r)
    peak = np.maximum.accumulate(eq_base)
    dd = eq_base/peak - 1.0  # drawdown series
    
    # --- Overlay 1: Drawdown-based deleveraging (monthly, contemporaneous) ---
    # levels: -5%→0.8x, -10%→0.6x, -15%→0.4x
    m_dd = np.ones_like(r)
    m_dd[dd <= -0.05] = 0.8
    m_dd[dd <= -0.10] = 0.6
    m_dd[dd <= -0.15] = 0.4
    
    r_dd = r * m_dd
    
    # --- Overlay 2: "Intramonth" circuit breaker proxy (monthly data) ---
    # Use SPY monthly shock as proxy; apply to NEXT month (t+1) to avoid look-ahead.
    # Trigger: SPY monthly <= -4% OR trailing 3m sum <= -6% -> next month's m_cb = 0.6
    spy_trailing_3m = pd.Series(spy_r).rolling(3).sum().values
    shock = (spy_r <= -0.04) | (spy_trailing_3m <= -0.06)
    m_cb = np.ones_like(r)
    # shift by 1 for next month application
    shock_shift = np.roll(shock, 1)
    shock_shift[0] = False
    m_cb[shock_shift] = 0.6
    
    r_cb_next = r * m_cb
    
    # --- Combined: apply both (priority min of multipliers) ---
    m_combined = np.minimum(m_dd, m_cb)
    r_combined = r * m_combined
    
    out = df.copy()
    out['DD'] = dd
    out['m_dd'] = m_dd
    out['m_cb_next'] = m_cb
    out['r_base'] = r
    out['r_dd'] = r_dd
    out['r_cb_next'] = r_cb_next
    out['r_combined'] = r_combined
    return out

def stats_from_returns(r):
    r = pd.Series(r).dropna()
    if len(r)==0: 
        return dict(AnnRet=np.nan, AnnVol=np.nan, Sharpe=np.nan, MDD=np.nan, Worst=np.nan, p5=np.nan)
    eq = (1+r).cumprod()
    annret = eq.iloc[-1]**(12/len(r)) - 1
    annvol = r.std() * np.sqrt(12)
    sharpe = annret/annvol if annvol>0 else np.nan
    peak = eq.cummax()
    dd = (eq/peak - 1).min()
    return dict(AnnRet=annret, AnnVol=annvol, Sharpe=sharpe, MDD=dd, Worst=r.min(), p5=r.quantile(0.05))

# Build overlays
o1 = add_overlays(p_spy)
o2 = add_overlays(p_spy_gld)

# Prepare summary table
def summarize(df, label):
    base = stats_from_returns(df['r_base'])
    ddov = stats_from_returns(df['r_dd'])
    cbov = stats_from_returns(df['r_cb_next'])
    cmb = stats_from_returns(df['r_combined'])
    rows = pd.DataFrame([base, ddov, cbov, cmb], index=[f"{label} - Base", f"{label} - DD", f"{label} - CB(next)", f"{label} - Combined"])
    return rows

summary = pd.concat([summarize(o1, "SPY Vol"), summarize(o2, "SPY Vol+GLD")], axis=0)
summary = summary.applymap(lambda x: round(x, 4) if isinstance(x, (int, float, np.floating)) else x)

import caas_jupyter_tools
caas_jupyter_tools.display_dataframe_to_user("월중 서킷브레이커(대용) + DD 디레버리징 성과 비교 (2022~)", summary)

# Plot cumulative curves: base vs combined for both portfolios
def plot_equity(df, title):
    r_base = df['r_base'].fillna(0).values
    r_comb = df['r_combined'].fillna(0).values
    eq_base = np.cumprod(1+r_base)
    eq_comb = np.cumprod(1+r_comb)
    x = df['Date'].values
    plt.figure()
    plt.plot(x, eq_base, label='Base')
    plt.plot(x, eq_comb, label='Combined')
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Cumulative")
    plt.legend()
    plt.show()

plot_equity(o1, "SPY Vol: Base vs Combined (2022~)")
plot_equity(o2, "SPY Vol+GLD: Base vs Combined (2022~)")

# Plot drawdowns for combined versions
def plot_dd(df, title):
    r_comb = df['r_combined'].fillna(0).values
    eq = np.cumprod(1+r_comb)
    peak = np.maximum.accumulate(eq)
    dd = eq/peak - 1
    x = df['Date'].values
    plt.figure()
    plt.plot(x, dd)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Drawdown")
    plt.show()

plot_dd(o1, "SPY Vol: Drawdown (Combined)")
plot_dd(o2, "SPY Vol+GLD: Drawdown (Combined)")